# Kasparro — Facebook Ads Analysis (Notebook)

**Author:** Amali Akshaya

This notebook performs a simple, non-AI analysis of the provided synthetic Facebook Ads dataset and produces three required deliverables:

- `insights.json`
- `creatives.json`
- `report.md`

It is structured to resemble the agentic pipeline (Planner → Data → Insights → Eval → Creatives) but implemented purely with pandas and simple logic.

## 0) Requirements

Run this cell to install dependencies if needed (Pandas & Matplotlib).

In [ ]:
!pip install --quiet pandas matplotlib nbformat

## 1) Planner

We will execute these steps:

1. Load and inspect data.
2. Run quick EDA (ROAS/CTR trends).
3. Identify low-CTR campaigns and ROAS drops.
4. Produce `insights.json`, `creatives.json`, and `report.md`.


In [ ]:
plan = [
    'Load data',
    'Quick EDA (head, missing, stats)',
    'Plot ROAS and CTR trends',
    'Identify low-CTR campaigns',
    'Generate insights and validate',
    'Create simple creatives',
    'Save outputs to /mnt/data/reports/'
]
print('Plan steps:')
for i, step in enumerate(plan,1):
    print(i, step)

## 2) Load data

In [ ]:

import pandas as pd, os
DATA_PATH = "/mnt/data/synthetic_fb_ads_undergarments.csv"
assert os.path.exists(DATA_PATH), f"Data file not found at {DATA_PATH}"
df = pd.read_csv(DATA_PATH)
# normalize column names
df.columns = [c.strip().lower() for c in df.columns]
# parse date if exists
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.shape


## 3) Quick EDA

In [ ]:

# Quick EDA
display(df.head(8))
print('\nData shape:', df.shape)
print('\nColumns:', df.columns.tolist())
print('\nMissing values (per column):')
print(df.isna().sum())
# Basic numeric summary for key metrics if present
metrics = [c for c in ['spend','impressions','clicks','ctr','purchases','revenue','roas'] if c in df.columns]
if metrics:
    display(df[metrics].describe().T)


## 4) Visualizations (ROAS trend & CTR by creative type)

In [ ]:

import matplotlib.pyplot as plt
# Create reports folder
OUT_DIR = "/mnt/data/reports"
os.makedirs(OUT_DIR, exist_ok=True)

# ROAS over time (daily mean)
if 'date' in df.columns and 'roas' in df.columns:
    roas_daily = df.groupby('date')['roas'].mean().dropna()
    plt.figure(figsize=(8,3.5))
    plt.plot(roas_daily.index, roas_daily.values)
    plt.title('Daily mean ROAS')
    plt.xlabel('Date')
    plt.ylabel('ROAS')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR,'roas_daily.png'))
    plt.show()
else:
    print('roas or date column missing; skipping ROAS trend plot.')

# CTR by creative type bar chart
if 'creative_type' in df.columns and 'ctr' in df.columns:
    ctr_by_type = df.groupby('creative_type')['ctr'].mean().sort_values(ascending=False)
    plt.figure(figsize=(6,3.5))
    plt.bar(ctr_by_type.index.astype(str), ctr_by_type.values)
    plt.title('Mean CTR by creative_type')
    plt.xlabel('creative_type')
    plt.ylabel('CTR')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR,'ctr_by_creative_type.png'))
    plt.show()
else:
    print('creative_type or ctr column missing; skipping CTR by creative_type plot.')


## 5) Generate Insights (rule-based)

In [ ]:

import json
insights = []

# 1) ROAS trend check
if 'date' in df.columns and 'roas' in df.columns:
    roas_daily = df.groupby('date')['roas'].mean().dropna()
    if len(roas_daily) >= 2:
        avg_daily_change = (roas_daily.diff().mean())
        hypothesis = 'ROAS has decreased over time' if avg_daily_change < 0 else 'ROAS is stable/increasing'
        insights.append({
            'hypothesis': hypothesis,
            'evidence': {
                'avg_daily_roas_change': round(float(avg_daily_change),4),
                'period_start': str(roas_daily.index.min().date()),
                'period_end': str(roas_daily.index.max().date())
            },
            'confidence': 0.8 if abs(avg_daily_change) > 0.01 else 0.6
        })

# 2) CTR by creative type
if 'creative_type' in df.columns and 'ctr' in df.columns:
    ctr_by_type = df.groupby('creative_type')['ctr'].mean().round(4).to_dict()
    insights.append({
        'hypothesis': 'CTR varies by creative_type',
        'evidence': ctr_by_type,
        'confidence': 0.9
    })

# 3) Low-CTR campaigns (bottom 5 by mean ctr)
if 'campaign_name' in df.columns and 'ctr' in df.columns:
    low_campaigns = df.groupby('campaign_name')['ctr'].mean().sort_values().head(5).round(4).to_dict()
    insights.append({
        'hypothesis': 'Top low-CTR campaigns identified',
        'evidence': low_campaigns,
        'confidence': 0.85
    })

# 4) Country-level ROAS
if 'country' in df.columns and 'roas' in df.columns:
    country_roas = df.groupby('country')['roas'].mean().round(4).to_dict()
    top_country = max(country_roas, key=country_roas.get) if country_roas else None
    insights.append({
        'hypothesis': 'Country-level ROAS',
        'evidence': {
            'country_roas': country_roas,
            'top_country': top_country
        },
        'confidence': 0.75
    })

# Save insights
insights_path = os.path.join(OUT_DIR,'insights.json')
with open(insights_path,'w') as f:
    json.dump(insights, f, indent=2)

print('Insights written to', insights_path)
insights


## 6) Validate Insights

In [ ]:

# Simple validation: mark accepted if confidence >= 0.7
validated = []
for i in insights:
    i['status'] = 'accepted' if i.get('confidence',0) >= 0.7 else 'needs_review'
    validated.append(i)

# Save validated insights as well
validated_path = os.path.join(OUT_DIR,'insights_validated.json')
with open(validated_path,'w') as f:
    json.dump(validated, f, indent=2)

print('Validated insights saved to', validated_path)
validated


## 7) Generate Creative Recommendations

In [ ]:

# Generate simple creatives (template-based) for low-CTR campaigns
creatives = []
low_campaigns = []
for ins in insights:
    if ins.get('hypothesis','').lower().startswith('top low-ctr') or 'low-ctr' in ins.get('hypothesis','').lower() or 'low-ctr' in str(ins.get('evidence','')).lower():
        # evidence is a dict of campaigns:ctr
        ev = ins.get('evidence',{})
        if isinstance(ev, dict):
            low_campaigns = list(ev.keys())

# If not found above, compute explicitly
if not low_campaigns and 'campaign_name' in df.columns and 'ctr' in df.columns:
    low_campaigns = df.groupby('campaign_name')['ctr'].mean().sort_values().head(3).index.tolist()

for c in low_campaigns:
    creatives.append({
        'campaign': c,
        'headline': f'{c} — Try shorter headline and clearer CTA',
        'description': 'Use product benefit first, add urgency (e.g., "Limited stock"), and test one visual variation.'
    })

# General creatives
creatives.append({
    'campaign': 'global',
    'headline': 'Soft, breathable comfort — Shop now',
    'description': 'Lead with comfort and product material; show close-up of fabric.'
})

# Save creatives
creatives_path = os.path.join(OUT_DIR,'creatives.json')
with open(creatives_path,'w') as f:
    json.dump(creatives, f, indent=2)

print('Creatives saved to', creatives_path)
creatives[:6]


## 8) Compose final `report.md`

In [ ]:

# Build a simple markdown report summarizing results
report_file = os.path.join(OUT_DIR,'report.md')
with open(report_file,'w') as f:
    f.write('# Kasparro - Facebook Ads Analysis Report\n\n')
    f.write('## Summary\n\n')
    f.write(f'- Rows: {df.shape[0]}\n')
    f.write(f'- Columns: {len(df.columns)}\n')
    if "date" in df.columns:
        f.write(f'- Date range: {str(df["date"].min().date())} to {str(df["date"].max().date())}\n')
    f.write('\n## Key Insights\n')
    for v in validated:
        f.write(f'- {v["hypothesis"]} (confidence: {v["confidence"]})\n')
        f.write(f'  - Evidence: {v["evidence"]}\n')
    f.write('\n## Creative Recommendations\n')
    for c in creatives:
        f.write(f'- {c["campaign"]}: {c["headline"]} — {c["description"]}\n')
    f.write('\n## How to reproduce\n')
    f.write('Run the cells in this notebook and inspect the files in `/mnt/data/reports/`\n')
print('Report written to', report_file)
report_file


## 9) Outputs

In [ ]:

# List produced files
import glob
for p in glob.glob(os.path.join(OUT_DIR,'*')):
    print('-', p.split('/')[-1])
print('\nAll done. Please check the /mnt/data/reports/ folder for outputs and images.\n')


## Submission checklist

1. Rename repository to `kasparro-agentic-fb-analyst-amali-akshaya`.
2. Add this notebook and the `reports/` folder to the repo.
3. Ensure `reports/insights.json`, `reports/creatives.json`, and `reports/report.md` exist.
4. Commit at least 3 commits, create a `v1.0` release tag, and submit the public GitHub link.

Good luck!